In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np

In [3]:
import os

In [4]:
import pickle
def save_obj(obj,path,name):
    with open(os.path.join(path, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from torch.utils.data import Dataset


ddi_label = load_obj("/mnt/c/Users/HP/Documents/drug_labels/training/ddi_test_510.pkl")
no_ddi_label = load_obj("/mnt/c/Users/HP/Documents/drug_labels/training/no_ddi_test_510.pkl")


for i in ddi_label:
    i.append(1)
for i in no_ddi_label:
    i.append(-1)

ddi = []

for i in ddi_label:
    ddi.append(i)
for i in no_ddi_label:
    ddi.append(i)

import random
random.shuffle(ddi)

ddi_np = np.array(ddi)

ddi_np.shape

np.save("/mnt/c/Users/HP/Documents/drug_labels/training/ddi_label_all",ddi_np)

np.save("/mnt/c/Users/HP/Documents/drug_labels/training/ddi_label_train",ddi_np[13000:,:])
np.save("/mnt/c/Users/HP/Documents/drug_labels/training/ddi_label_test",ddi_np[:13000,:])

training_data = np.load("/mnt/c/Users/HP/Documents/drug_labels/training/scibert_1/scibert_data.npy")

training_data_torch = np.moveaxis(training_data, -1, 1)


np.all(training_data[0,0,:] == training_data_torch[0,:,0])

for i in range(training_data.shape[0]):
    for j in range(training_data.shape[1]):
        if np.all(training_data[i,j,:] == training_data_torch[i,:,j]) :
            pass
        else:
            print(i,j)

type(training_data) == np.ndarray

In [6]:
#feature file is the same for training and testing, the labeling file is different for training and testing,
#have different index
class DDI_Dataset(Dataset):
    def __init__(self, feature_file, label_file, transform=False, target_transform=False):
        if type(feature_file) == str and os.path.exists(feature_file):
            self.features = np.load(feature_file)
        elif type(feature_file) == np.ndarray:
            self.features = feature_file
        else:
            self.features = feature_file
            
        if type(label_file) == str and os.path.exists(label_file):
            self.labels = np.load(label_file)
        elif type(label_file) == np.ndarray:
            self.labels = label_file
        else:
            self.labels = label_file

        self.transform = transform
        self.target_transform = target_transform
        _, self.counts = np.unique(self.labels[:,2], return_index=False, return_inverse=False, return_counts=True, axis=None)
        if target_transform == True:
            self.labels[:,2].sort()
            

    def __len__(self):
        if self.target_transform == False:
            return self.labels.shape[0]
        else:
            
            return self.counts.min()*2

    def __getitem__(self, idx):
        if self.target_transform == False:

            id1,id2,mark = self.labels[idx]
            if mark == 1: 
                y = 1
            else:
                y = 0

            fea = np.concatenate((self.features[id1], self.features[id2]))
            if self.transform == True:
                fea = np.moveaxis(fea,-1,0)
            return fea,y
        
        else:
            if idx %2 == 0:
                id1,id2,mark = self.labels[-idx//2-1]
            else:
                new_idx = np.random.randint(0,self.counts.max())
                id1,id2,mark = self.labels[new_idx]
            
            if mark == 1: 
                y = 1
            else:
                y = 0

            fea = np.concatenate((self.features[id1], self.features[id2]))
            if self.transform == True:
                fea = np.moveaxis(fea,-1,0)
            return fea,y
        

In [7]:
train_dataset = DDI_Dataset("/mnt/c/Users/HP/Documents/drug_labels/training/scibert_2/scibert_data.npy",\
                            "/mnt/c/Users/HP/Documents/drug_labels/training/ddi_label_train.npy",transform = True, target_transform= True)
test_dataset = DDI_Dataset("/mnt/c/Users/HP/Documents/drug_labels/training/scibert_2/scibert_data.npy",\
                            "/mnt/c/Users/HP/Documents/drug_labels/training/ddi_label_test.npy", transform = True)

In [8]:
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

>>> loss = nn.CrossEntropyLoss()
>>> input = torch.randn(3, 5, requires_grad=True)
>>> target = torch.empty(3, dtype=torch.long).random_(5)
>>> output = loss(input, target)
>>> output.backward()

next(iter(train_dataloader))[0].shape

In [9]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(768, 128, 3)
        #self.pool = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(128, 16, 3)
        self.fc1 = nn.Linear(16*6 , 16)
        #self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1, 16*6)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()

Using cpu device


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [12]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    positive_correct = 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            positive_correct += (np.logical_and((y == 1),(pred.argmax(1) == y))).type(torch.float).sum().item()
    precision = positive_correct/correct
    recall = positive_correct/2552
    test_loss /= size
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(f"Precision {(100*precision):>0.1f}%, Recall: {(100*recall):>0.1f}% \n")

In [15]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 0.693189  [    0/47778]
loss: 0.693256  [ 6400/47778]
loss: 0.692821  [12800/47778]
loss: 0.693148  [19200/47778]
loss: 0.693000  [25600/47778]
loss: 0.693169  [32000/47778]
loss: 0.693092  [38400/47778]
loss: 0.693221  [44800/47778]
Test Error: 
 Accuracy: 19.6%, Avg loss: 0.010880 

Precision 100.0%, Recall: 100.0% 

Epoch 2
-------------------------------
loss: 0.693115  [    0/47778]
loss: 0.693148  [ 6400/47778]
loss: 0.693171  [12800/47778]
loss: 0.693349  [19200/47778]
loss: 0.693123  [25600/47778]
loss: 0.693116  [32000/47778]
loss: 0.693167  [38400/47778]
loss: 0.693147  [44800/47778]
Test Error: 
 Accuracy: 19.6%, Avg loss: 0.010880 

Precision 100.0%, Recall: 100.0% 

Epoch 3
-------------------------------
loss: 0.693095  [    0/47778]
loss: 0.693311  [ 6400/47778]
loss: 0.693180  [12800/47778]
loss: 0.693117  [19200/47778]
loss: 0.693148  [25600/47778]
loss: 0.693277  [32000/47778]
loss: 0.693202  [38400/47778]
loss: 0.693127  

In [ ]:
torch.save(model.state_dict(), "/mnt/c/Users/HP/Documents/drug_labels/training/fully_connected/model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("/mnt/c/Users/HP/Documents/drug_labels/training/fully_connected/model.pth"))

In [ ]:
classes = [
"no DDI",
    "DDI"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')